### Part IV - Generalized Linear Model, Bagging, Boosting and Stacking

In [323]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats

In [324]:
import matplotlib.pyplot as plt
%matplotlib inline 

In [325]:
import h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator
from h2o.grid.grid_search import H2OGridSearch
from __future__ import print_function
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,1 day 8 hours 5 mins
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.3
H2O cluster version age:,25 days
H2O cluster name:,H2O_from_python_chitr_8vg90n
H2O cluster total nodes:,1
H2O cluster free memory:,2.308 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [326]:
# Import data

In [327]:
data = h2o.import_file('OnlineNewsPopularity/OnlineNewsPopularity.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [328]:
data.head()

url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
http://mashable.com/2013/01/07/amazon-instant-video-browser/,731,12,219,0.663594,1,0.815385,4,2,1,0,4.68037,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,496,496,496,1,0,0,0,0,0,0,0,0.500331,0.378279,0.0400047,0.0412626,0.0401225,0.521617,0.092562,0.0456621,0.0136986,0.769231,0.230769,0.378636,0.1,0.7,-0.35,-0.6,-0.2,0.5,-0.1875,0,0.1875,593
http://mashable.com/2013/01/07/ap-samsung-sponsored-tweets/,731,9,255,0.604743,1,0.791946,3,1,1,0,4.91373,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.799756,0.0500467,0.0500963,0.0501007,0.0500007,0.341246,0.148948,0.0431373,0.0156863,0.733333,0.266667,0.286915,0.0333333,0.7,-0.11875,-0.125,-0.1,0,0,0.5,0,711
http://mashable.com/2013/01/07/apple-40-billion-app-downloads/,731,9,211,0.57513,1,0.663866,3,1,1,0,4.39336,6,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,918,918,918,1,0,0,0,0,0,0,0,0.217792,0.0333345,0.0333514,0.0333335,0.682188,0.702222,0.323333,0.056872,0.00947867,0.857143,0.142857,0.495833,0.1,1,-0.466667,-0.8,-0.133333,0,0,0.5,0,1500
http://mashable.com/2013/01/07/astronaut-notre-dame-bcs/,731,9,531,0.503788,1,0.665635,9,0,1,0,4.4049,7,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.0285732,0.4193,0.494651,0.0289047,0.0285716,0.42985,0.100705,0.0414313,0.0207156,0.666667,0.333333,0.385965,0.136364,0.8,-0.369697,-0.6,-0.166667,0,0,0.5,0,1200
http://mashable.com/2013/01/07/att-u-verse-apps/,731,13,1072,0.415646,1,0.54089,19,19,20,0,4.68284,7,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,545,16000,3151.16,1,0,0,0,0,0,0,0,0.0286328,0.0287936,0.0285752,0.0285717,0.885427,0.513502,0.281003,0.0746269,0.0121269,0.860215,0.139785,0.411127,0.0333333,1,-0.220192,-0.5,-0.05,0.454545,0.136364,0.0454545,0.136364,505
http://mashable.com/2013/01/07/beewi-smart-toys/,731,10,370,0.559889,1,0.698198,2,2,0,0,4.35946,9,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8500,8500,8500,1,0,0,0,0,0,0,0,0.0222453,0.306718,0.0222313,0.0222243,0.626582,0.437409,0.0711842,0.0297297,0.027027,0.52381,0.47619,0.35061,0.136364,0.6,-0.195,-0.4,-0.1,0.642857,0.214286,0.142857,0.214286,855
http://mashable.com/2013/01/07/bodymedia-armbandgets-update/,731,8,960,0.418163,1,0.549834,21,20,20,0,4.65417,10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,545,16000,3151.16,1,0,0,0,0,0,0,0,0.0200817,0.114705,0.0200244,0.0200153,0.825173,0.51448,0.268303,0.0802083,0.0166667,0.827957,0.172043,0.402039,0.1,1,-0.224479,-0.5,-0.05,0,0,0.5,0,556
http://mashable.com/2013/01/07/canon-poweshot-n/,731,12,989,0.433574,1,0.572108,20,20,20,0,4.6178,9,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,545,16000,3151.16,1,0,0,0,0,0,0,0,0.0222244,0.150733,0.243435,0.0222236,0.561384,0.543474,0.298613,0.0839232,0.0151668,0.846939,0.153061,0.42772,0.1,1,-0.242778,-0.5,-0.05,1,0.5,0.5,0.5,891
http://mashable.com/2013/01/07/car-of-the-future-infographic/,731,11,97,0.670103,1,0.836735,2,0,0,0,4.85567,7,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.45825,0.0289794,0.0286619,0.0296959,0.454412,0.538889,0.161111,0.0309278,0.0206186,0.6,0.4,0.566667,0.4,0.8,-0.125,-0.125,-0.125,0.125,0,

In [329]:
def convertor(x):
    if(x > 1400):
        return 1
    else:
        return 0

In [330]:
NumberOfShares = data['shares']

In [331]:
data = data.concat(NumberOfShares, axis=1)

In [332]:
data['shares2'] = (data['shares'] > 1400).ifelse(1,0)

In [333]:
data.head()

url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,shares0,shares2
http://mashable.com/2013/01/07/amazon-instant-video-browser/,731,12,219,0.663594,1,0.815385,4,2,1,0,4.68037,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,496,496,496,1,0,0,0,0,0,0,0,0.500331,0.378279,0.0400047,0.0412626,0.0401225,0.521617,0.092562,0.0456621,0.0136986,0.769231,0.230769,0.378636,0.1,0.7,-0.35,-0.6,-0.2,0.5,-0.1875,0,0.1875,593,593,0
http://mashable.com/2013/01/07/ap-samsung-sponsored-tweets/,731,9,255,0.604743,1,0.791946,3,1,1,0,4.91373,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.799756,0.0500467,0.0500963,0.0501007,0.0500007,0.341246,0.148948,0.0431373,0.0156863,0.733333,0.266667,0.286915,0.0333333,0.7,-0.11875,-0.125,-0.1,0,0,0.5,0,711,711,0
http://mashable.com/2013/01/07/apple-40-billion-app-downloads/,731,9,211,0.57513,1,0.663866,3,1,1,0,4.39336,6,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,918,918,918,1,0,0,0,0,0,0,0,0.217792,0.0333345,0.0333514,0.0333335,0.682188,0.702222,0.323333,0.056872,0.00947867,0.857143,0.142857,0.495833,0.1,1,-0.466667,-0.8,-0.133333,0,0,0.5,0,1500,1500,1
http://mashable.com/2013/01/07/astronaut-notre-dame-bcs/,731,9,531,0.503788,1,0.665635,9,0,1,0,4.4049,7,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.0285732,0.4193,0.494651,0.0289047,0.0285716,0.42985,0.100705,0.0414313,0.0207156,0.666667,0.333333,0.385965,0.136364,0.8,-0.369697,-0.6,-0.166667,0,0,0.5,0,1200,1200,0
http://mashable.com/2013/01/07/att-u-verse-apps/,731,13,1072,0.415646,1,0.54089,19,19,20,0,4.68284,7,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,545,16000,3151.16,1,0,0,0,0,0,0,0,0.0286328,0.0287936,0.0285752,0.0285717,0.885427,0.513502,0.281003,0.0746269,0.0121269,0.860215,0.139785,0.411127,0.0333333,1,-0.220192,-0.5,-0.05,0.454545,0.136364,0.0454545,0.136364,505,505,0
http://mashable.com/2013/01/07/beewi-smart-toys/,731,10,370,0.559889,1,0.698198,2,2,0,0,4.35946,9,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8500,8500,8500,1,0,0,0,0,0,0,0,0.0222453,0.306718,0.0222313,0.0222243,0.626582,0.437409,0.0711842,0.0297297,0.027027,0.52381,0.47619,0.35061,0.136364,0.6,-0.195,-0.4,-0.1,0.642857,0.214286,0.142857,0.214286,855,855,0
http://mashable.com/2013/01/07/bodymedia-armbandgets-update/,731,8,960,0.418163,1,0.549834,21,20,20,0,4.65417,10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,545,16000,3151.16,1,0,0,0,0,0,0,0,0.0200817,0.114705,0.0200244,0.0200153,0.825173,0.51448,0.268303,0.0802083,0.0166667,0.827957,0.172043,0.402039,0.1,1,-0.224479,-0.5,-0.05,0,0,0.5,0,556,556,0
http://mashable.com/2013/01/07/canon-poweshot-n/,731,12,989,0.433574,1,0.572108,20,20,20,0,4.6178,9,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,545,16000,3151.16,1,0,0,0,0,0,0,0,0.0222244,0.150733,0.243435,0.0222236,0.561384,0.543474,0.298613,0.0839232,0.0151668,0.846939,0.153061,0.42772,0.1,1,-0.242778,-0.5,-0.05,1,0.5,0.5,0.5,891,891,0
http://mashable.com/2013/01/07/car-of-the-future-infographic/,731,11,97,0.670103,1,0.836735,2,0,0,0,4.85567,7,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.45825,0.0289794,0.0286619,0.0296959,0.454412,0.538889,0.161111,0.030927

In [334]:
predictor = [[
 'timedelta',
 'n_tokens_title',
 'n_tokens_content',
 'n_unique_tokens',
 'n_non_stop_words',
 'n_non_stop_unique_tokens',
 'num_hrefs',
 'num_self_hrefs',
 'num_imgs',
 'num_videos',
 'average_token_length',
 'num_keywords',
 'data_channel_is_lifestyle',
 'data_channel_is_entertainment',
 'data_channel_is_bus',
 'data_channel_is_socmed',
 'data_channel_is_tech',
 'data_channel_is_world',
 'kw_min_min',
 'kw_max_min',
 'kw_avg_min',
 'kw_min_max',
 'kw_max_max',
 'kw_avg_max',
 'kw_min_avg',
 'kw_max_avg',
 'kw_avg_avg',
 'self_reference_min_shares',
 'self_reference_max_shares',
 'self_reference_avg_sharess',
 'weekday_is_monday',
 'weekday_is_tuesday',
 'weekday_is_wednesday',
 'weekday_is_thursday',
 'weekday_is_friday',
 'weekday_is_saturday',
 'weekday_is_sunday',
 'is_weekend',
 'LDA_00',
 'LDA_01',
 'LDA_02',
 'LDA_03',
 'LDA_04',
 'global_subjectivity',
 'global_sentiment_polarity',
 'global_rate_positive_words',
 'global_rate_negative_words',
 'rate_positive_words',
 'rate_negative_words',
 'avg_positive_polarity',
 'min_positive_polarity',
 'max_positive_polarity',
 'avg_negative_polarity',
 'min_negative_polarity',
 'max_negative_polarity',
 'title_subjectivity',
 'title_sentiment_polarity',
 'abs_title_subjectivity',
 'abs_title_sentiment_polarity']]

In [335]:
target = 'shares2'

In [336]:
# Split dataset into train and test

In [337]:
train, test = data.split_frame(ratios=[0.7])

In [338]:
# Generate GLM model

In [339]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [340]:
glm_classifier = H2OGeneralizedLinearEstimator(family="binomial", nfolds=10, alpha=0.5)

In [341]:
glm_classifier.train(x = predictor, y = target, training_frame=train)

H2OTypeError: Argument `x` should be a None | integer | string | list(string | integer) | set(integer | string), got list [['timedelta', 'n_tokens_title', 'n_tokens_content', 'n_unique_tokens', 'n_non_stop_words', 'n_non_stop_unique_tokens', 'num_hrefs', 'num_self_hrefs', 'num_imgs', 'num_videos', 'average_token_length', 'num_keywords', 'data_channel_is_lifestyle', 'data_channel_is_entertainment', 'data_channel_is_bus', 'data_channel_is_socmed', 'data_channel_is_tech', 'data_channel_is_world', 'kw_min_min', 'kw_max_min', 'kw_avg_min', 'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg', 'kw_avg_avg', 'self_reference_min_shares', 'self_reference_max_shares', 'self_reference_avg_sharess', 'weekday_is_monday', 'weekday_is_tuesday', 'weekday_is_wednesday', 'weekday_is_thursday', 'weekday_is_friday', 'weekday_is_saturday', 'weekday_is_sunday', 'is_weekend', 'LDA_00', 'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 'global_subjectivity', 'global_sentiment_polarity', 'global_rate_pos

In [ ]:
glm_classifier.coef()

In [ ]:
print(glm_classifier.coef_norm())

In [ ]:
glm_classifier._model_json['output']['coefficients_table']

In [ ]:
preds  = glm_classifier.predict(test)

In [ ]:
glm_classifier.model_performance(test)

In [ ]:
glm_classifier.confusion_matrix(metrics="accuracy")

In [ ]:
glm_classifier.logloss(train=True)

In [ ]:
glm_classifier.accuracy()

In [ ]:
# So, the accuracy using GLM model on train data is 48% and on test data is 65%

In [ ]:
## QQ Plot

In [ ]:
import statsmodels.api as sm

In [ ]:
res = glm_classifier.residual_deviance()

In [ ]:
res

In [ ]:
glm_classifier.residual_degrees_of_freedom()

In [ ]:
# Accuracy of model is 50%

In [ ]:
# ROC Curve

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
# Bagging , Boosting, Stacking

In [ ]:
train[target] = train[target].asfactor()
test[target] = test[target].asfactor()

In [ ]:
n_folds = 5

In [ ]:
# Train and cross-validate a GBM
my_gbm = H2OGradientBoostingEstimator(distribution="bernoulli",
                                      ntrees=10,
                                      max_depth=3,
                                      min_rows=2,
                                      learn_rate=0.2,
                                      nfolds=nfolds,
                                      fold_assignment="Modulo",
                                      keep_cross_validation_predictions=True,
                                      seed=1)
my_gbm.train(x=predictor, y=target, training_frame=train)

In [ ]:
# Train and cross-validate a RF
my_rf = H2ORandomForestEstimator(ntrees=50,
                                 nfolds=nfolds,
                                 fold_assignment="Modulo",
                                 keep_cross_validation_predictions=True,
                                 seed=1)
my_rf.train(x=predictor, y=target, training_frame=train)

In [ ]:
# Train a stacked ensemble using the GBM and GLM above
ensemble = H2OStackedEnsembleEstimator(model_id="my_ensemble_binomial",
                                       base_models=[my_gbm, my_rf])
ensemble.train(x=predictor, y=target, training_frame=train)

In [ ]:
# Eval ensemble performance on the test data
perf_stack_test = ensemble.model_performance(test)

In [ ]:
# Compare to base learner performance on the test set
perf_gbm_test = my_gbm.model_performance(test)
perf_rf_test = my_rf.model_performance(test)
baselearner_best_auc_test = max(perf_gbm_test.auc(), perf_rf_test.auc())
stack_auc_test = perf_stack_test.auc()
print("Best Base-learner Test AUC:  {0}".format(baselearner_best_auc_test))
print("Ensemble Test AUC:  {0}".format(stack_auc_test))

In [ ]:
# Generate predictions on a test set (if neccessary)
pred = ensemble.predict(test)

In [ ]:
ensemble.confusion_matrix(metrics="accuracy")

In [ ]:
# Accuracy using ensemble is 52%

In [ ]:
# Specify GBM hyperparameters for the grid
hyper_params = {"learn_rate": [0.01, 0.03],
                "max_depth": [3, 4, 5, 6, 9],
                "sample_rate": [0.7, 0.8, 0.9, 1.0],
                "col_sample_rate": [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]}
search_criteria = {"strategy": "RandomDiscrete", "max_models": 3, "seed": 1}

In [ ]:
# Train the grid
grid = H2OGridSearch(model=H2OGradientBoostingEstimator(ntrees=10,
                                                        seed=1,
                                                        nfolds=nfolds,
                                                        fold_assignment="Modulo",
                                                        keep_cross_validation_predictions=True),
                     hyper_params=hyper_params,
                     search_criteria=search_criteria,
                     grid_id="gbm_grid_binomial")
grid.train(x=predictor, y=target, training_frame=train)

In [ ]:
# Train a stacked ensemble using the GBM grid
ensemble = H2OStackedEnsembleEstimator(model_id="my_ensemble_gbm_grid_binomial",
                                       base_models=grid.model_ids)
ensemble.train(x=predictor, y=target, training_frame=train)

In [ ]:
# Eval ensemble performance on the test data
perf_stack_test = ensemble.model_performance(test)

In [ ]:
# Compare to base learner performance on the test set
baselearner_best_auc_test = max([h2o.get_model(model).model_performance(test_data=test).auc() for model in grid.model_ids])
stack_auc_test = perf_stack_test.auc()
print("Best Base-learner Test AUC:  {0}".format(baselearner_best_auc_test))
print("Ensemble Test AUC:  {0}".format(stack_auc_test))

In [ ]:
# Generate predictions on a test set (if neccessary)
pred = ensemble.predict(test)

In [ ]:
ensemble.confusion_matrix(metrics="accuracy")

In [ ]:
# Accuracy after using hyper parameters is 70%

In [ ]:
# Generalized Linear Model -- Version I

In [ ]:
import glmnet_python
from glmnet import glmnet
from glmnetPlot import glmnetPlot
from glmnetPrint import glmnetPrint
from glmnetCoef import glmnetCoef
from glmnetPredict import glmnetPredict
from cvglmnet import cvglmnet
from cvglmnetCoef import cvglmnetCoef
from cvglmnetPlot import cvglmnetPlot
from cvglmnetPredict import cvglmnetPredict

In [369]:
X_data = np.ndarray(data[[
 'timedelta',
 'n_tokens_title',
 'n_tokens_content',
 'n_unique_tokens',
 'n_non_stop_words',
 'n_non_stop_unique_tokens',
 'num_hrefs',
 'num_self_hrefs',
 'num_imgs',
 'num_videos',
 'average_token_length',
 'num_keywords',
 'data_channel_is_lifestyle',
 'data_channel_is_entertainment',
 'data_channel_is_bus',
 'data_channel_is_socmed',
 'data_channel_is_tech',
 'data_channel_is_world',
 'kw_min_min',
 'kw_max_min',
 'kw_avg_min',
 'kw_min_max',
 'kw_max_max',
 'kw_avg_max',
 'kw_min_avg',
 'kw_max_avg',
 'kw_avg_avg',
 'self_reference_min_shares',
 'self_reference_max_shares',
 'self_reference_avg_sharess',
 'weekday_is_monday',
 'weekday_is_tuesday',
 'weekday_is_wednesday',
 'weekday_is_thursday',
 'weekday_is_friday',
 'weekday_is_saturday',
 'weekday_is_sunday',
 'is_weekend',
 'LDA_00',
 'LDA_01',
 'LDA_02',
 'LDA_03',
 'LDA_04',
 'global_subjectivity',
 'global_sentiment_polarity',
 'global_rate_positive_words',
 'global_rate_negative_words',
 'rate_positive_words',
 'rate_negative_words',
 'avg_positive_polarity',
 'min_positive_polarity',
 'max_positive_polarity',
 'avg_negative_polarity',
 'min_negative_polarity',
 'max_negative_polarity',
 'title_subjectivity',
 'title_sentiment_polarity',
 'abs_title_subjectivity',
 'abs_title_sentiment_polarity']])

ValueError: sequence too large; cannot be greater than 32

In [367]:
X_data = X_data.as_data_frame

In [350]:
y = data['shares2']

In [368]:
glm_fit = glmnet(x = 'X', y='y', family = 'binomial')

ValueError: x input must be a scipy float64 ndarray